<a href="https://colab.research.google.com/github/matheusvazdata/dados-lab/blob/main/Importing_flat_files_from_the_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de Dados da Web

### CSV

In [1]:
# Importação das bibliotecas necessárias
from urllib.request import urlretrieve
import pandas as pd
import requests

In [2]:
# Busca e salvamento do arquivo
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'
urlretrieve(url, 'winequality-white.csv') # Agora ele está na pasta content do Colab

('winequality-white.csv', <http.client.HTTPMessage at 0x7bcfb9b84210>)

In [3]:
# Carregamento do arquivo como DataFrame
df_wine = pd.read_csv('winequality-white.csv', sep=';') # Lembrar que o separador deste é o ";"

# Visualização das 5 primeiras linhas
df_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


##### Outro exemplo com um arquivo hospedado no meu github

In [4]:
# URL com os dados em CSV do dataset de filmes que ficaram no top 10 da Netflix
url_netflix_top_10 = 'https://raw.githubusercontent.com/matheusvazdata/ia-desafio-aed/refs/heads/main/data/daily_top_10_netflix%20.csv'

# Salvamento do arquivo na pasta content como 'netflix_top_10.csv'
urlretrieve(url_netflix_top_10, 'netflix_top_10.csv')

('netflix_top_10.csv', <http.client.HTTPMessage at 0x7bcfb9743bd0>)

In [5]:
# Criação do df e visualização das 3 primeiras linhas
df_netflix_top_10 = pd.read_csv('/content/netflix_top_10.csv')
df_netflix_top_10.head(3)

,As of,Rank,Year to Date Rank,Last Week Rank,Title,Type,Netflix Exclusive,Netflix Release Date,Days In Top 10,Viewership Score
0,2020-04-01,1,1,1,"Tiger King: Murder, Mayhem …",TV Show,Yes,"Mar 20, 2020",9,90
1,2020-04-01,2,2,-,Ozark,TV Show,Yes,"Jul 21, 2017",5,45
2,2020-04-01,3,3,2,All American,TV Show,NaN,"Mar 28, 2019",9,76


### JSON

In [6]:
# Definicação da URL do JSON
url_json = 'https://jsonplaceholder.typicode.com/users'

# Requisição HTTP para obter os dados
response = requests.get(url_json)

In [7]:
# Verificação da requisição (se foi bem-sucedida ou não)
if response.status_code == 200:
    # Conversão dos dados para JSON
    data = response.json()

    # Carregamento dos dados em um DataFrame do pandas
    df_json = pd.DataFrame(data)

    # Exibição das 2 primeiras linhas do DataFrame
    display(df_json.head(2))
else:
    print(f"Erro ao acessar a URL: {response.status_code}")

,id,name,username,email,address,phone,website,company
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu..."
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac..."


> Note que, as colunas `address` e `company`, ainda estão no formato de dicionário, ou seja, os valores dessas colunas são outras estruturas `chave-valor`. Para resolver isso, o pandas tem uma função chamada `json_normalize` que "explode" todas essas chaves em novas colunas

In [8]:
# Explosão das colunas manualmente e adição dos prefixos
df_address = pd.json_normalize(df_json['address']).add_prefix('address_')
df_company = pd.json_normalize(df_json['company']).add_prefix('company_')

In [9]:
# Concatenação com o DataFrame original e remoção das colunas antigas
df_json_exploded = pd.concat([df_json.drop(columns=['address', 'company']), df_address, df_company], axis=1)

# Visualização das 2 primeiras linhas
df_json_exploded.head(2)

,id,name,username,email,phone,website,address_street,address_suite,address_city,address_zipcode,address_geo.lat,address_geo.lng,company_name,company_catchPhrase,company_bs
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.3159,81.1496,Romaguera-Crona,Multi-layered client-server neural-net,harness real-time e-markets
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.9509,-34.4618,Deckow-Crist,Proactive didactic contingency,synergize scalable supply-chains
